In [2]:
import numpy as np
from scipy.interpolate import BSpline

from mpl_toolkits import mplot3d

import matplotlib.pyplot as plt

%matplotlib qt



/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Cubic B-Spline interpolation with equidistant knots


In [56]:
def basis_function(t, derivate = 0):
    assert derivate in [0,1,2], "derivate must be 0, 1 or 2"

    if derivate == 0:
      t_ = np.array([1, t, t**2, t**3])
    if derivate == 1:
       t_ = np.array([0, 1, 2*t, 3*t**2])
    if derivate == 2:
       t_ = np.array(([0,0, 2, 6* t]))
    
    feature_matrix = 1/6* np.array([[1, 4, 1, 0],
                               [-3,0 ,3,0],
                                 [3,-6,3,0],
                                 [-1,3,-3,1]])
    
    return t_ @ feature_matrix


In [57]:
ts = np.linspace(0, 1, 100)
basis = np.zeros((100, 4))
for i, t in enumerate(ts):
    basis[i] = basis_function(t)

plt.plot(ts, basis[:,0], label="Basis 0")
plt.plot(ts, basis[:,1], label="Basis 1")
plt.plot(ts, basis[:,2], label="Basis 2")
plt.plot(ts, basis[:,3], label="Basis 3")


In [61]:
# vectorize the basis function
def basis_function_mat(ts, n_control_points, n_knots, derivate = 0):
    mat = np.zeros((ts.shape[0], n_control_points))
    for i, t in enumerate(ts):
        offset = max(min(int(np.floor(t)), n_knots -1),0)
        u = t - offset
        mat[i][offset: offset + 4] = basis_function(u, derivate = derivate)
    
    return mat


In [74]:
control_points = np.array([[0,-10], [-2,-1], [-3,3],[-2,6], [0,5], [2,6], [3,3], [2,-1], [3,1], [4,0], [5,-10], [0,-20]])
knots = [0,1,2,3,4,5,6,7]

curve = np.zeros((100, 2))
ts = np.linspace(-2, 10, 100)

basis = basis_function_mat(ts,12, 8)
curve = basis @ control_points


plt.plot(curve[:,0], curve[:,1], label="B-spline curve")
plt.plot(control_points[:,0], control_points[:,1], "ro-")


In [72]:

def spline_eval(control_points, num_samples, derivate = 0):
    n_knots = control_points.shape[0] - 4

    ts = np.linspace(0, n_knots, num_samples)
    basis = basis_function_mat(ts, control_points.shape[0], n_knots, derivate = derivate)
    curve = basis @ control_points

    return curve



### Parameteroptimition for cubic B-Spline with obstacles

In [106]:
import casadi as cas

# Decision variables
n_control_points = 12
dim_control_points = 3
n_knots = n_control_points - 4
n_samples = 50


control_points = cas.SX.sym("control_points", n_control_points, dim_control_points)
dec_vars = cas.vertcat(cas.vec(control_points))


# Define Curve points with respect to control points and basis functions
curve = spline_eval(control_points, n_samples)
dcurve = spline_eval(control_points, n_samples, derivate =1)
ddcurve = spline_eval(control_points, n_samples, derivate = 2)

# Define constraints
start_point = np.array([0, 0 , 0])

end_point = np.array([30, 30, 0])

cons = cas.SX([])
lbg = []
ubg = []

cons = cas.vertcat(cons, (curve[0,0] - start_point[0]) ** 2 + (curve[0,1] - start_point[1]) ** 2 + (curve[0,2] - start_point[2]) ** 2)
lbg = np.concatenate((lbg, [0]))
ubg = np.concatenate((ubg, [0.1]))

cons = cas.vertcat(cons, (curve[-1,0] - end_point[0]) ** 2 + (curve[-1,1] - end_point[1]) ** 2 + (curve[-1,2] - end_point[2]) ** 2)
lbg = np.concatenate((lbg, [0]))
ubg = np.concatenate((ubg, [0.1]))

# Define Obstacle constraint
obstacle_positions = np.array([[3, 3, 0], [2,20,0], [20,2,0], [15,15,0]])
obstacle_radii = np.array([3,5,5,1])
for i in range(n_samples):
    for obstacle_position, obstacle_radius in zip(obstacle_positions, obstacle_radii):
        cons = cas.vertcat(cons, (curve[i,0] - obstacle_position[0]) ** 2 + (curve[i,1] - obstacle_position[1]) ** 2 + (curve[i,2] - obstacle_position[2]) ** 2 - (obstacle_radius + 1)  ** 2)
        lbg = np.concatenate((lbg, [0]))
        ubg = np.concatenate((ubg, [cas.inf]))

# Define cost function to minimize the length of the B-spline curve
cost = 0
for i in range(n_samples-1):
    cost = cost + (curve[i,0] - curve[i+1,0]) ** 2 + (curve[i,1] - curve[i+1,1]) ** 2 + (curve[i,2] - curve[i+1,2]) ** 2

cost += cas.sum1(cas.sum2(ddcurve**2))



# Create Optimization problem
nlp = {"x": dec_vars, "f": cost, "g": cons}
ipop_options = {"ipopt.print_level": 3, "ipopt.max_iter": 1000, "ipopt.tol": 1e-3, "print_time": 0, "ipopt.acceptable_tol": 1e-3, "ipopt.acceptable_obj_change_tol": 1e-3, "ipopt.hessian_approximation": "limited-memory"}

solver = cas.nlpsol("solver", "ipopt", nlp, ipop_options)

# Solve Optimization problem
res = solver(lbg=lbg, ubg=ubg)
sol_control_points = np.array(res["x"].reshape((n_control_points, dim_control_points)))

# Plot the B-spline using numpy and scipy
curve = spline_eval(sol_control_points, n_samples*10)
print(curve.shape)

# Plot in 3D
fig = plt.figure()
ax = plt.axes(projection='3d')

for obstacle_position, obstacle_radius in zip(obstacle_positions, obstacle_radii):
    u = np.linspace(0, np.pi, 30)
    v = np.linspace(0, 2 * np.pi, 30)

    x = np.outer(np.sin(u), np.sin(v)) * obstacle_radius + obstacle_position[0]
    y = np.outer(np.sin(u), np.cos(v)) * obstacle_radius + obstacle_position[1]
    z = np.outer(np.cos(u), np.ones_like(v)) * obstacle_radius + obstacle_position[2]

    ax.plot_wireframe(x,y,z, color = "red")


ax.plot3D(curve[:,0], curve[:,1], curve[:,2], label="B-spline curve")


   


plt.show()


Total number of variables............................:       36
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:      202
        inequality constraints with only lower bounds:      200
   inequality constraints with lower and upper bounds:        2
        inequality constraints with only upper bounds:        0


Number of Iterations....: 132

                                   (scaled)                 (unscaled)
Objective...............:   1,0485369764105636e+02    1,0485369764105636e+02
Dual infeasibility......:   6,9631481520311781e-05    6,9631481520311781e-05
Constraint violation....:   2,8568167276254997e-10    2,8568167276254997e-10
Variable bound violation:   0,0000000000000000e+00    0,0000000000000000e+00
Complementar

### Trajectory Optimization using Neural Radiance Field (NRF as cost function)

### Helpful Links
- https://gist.github.com/jgillis/54767ae9e38dca3dfcb9144fb4eb4398 
- https://www.youtube.com/watch?v=jvPPXbo87ds